### Imports

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import sklearn as sk
from utils import print_dictionary_head, load_from_file

### Load data

In [2]:
train = load_from_file('data/train.csv','Id')

(9557, 142)


In [3]:
test = load_from_file('data/test.csv','Id')

(23856, 141)


In [4]:
train.head(2)

,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
Id,,,,,,,,,,,,,,,,,,,,,
ID_279628684,190000.0,0,3,0,1,1,0,NaN,0,1,...,100,1849,1,100,0,1.0,0.0,100.0,1849,4
ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,1,...,144,4489,1,144,0,1.0,64.0,144.0,4489,4


In [5]:
test.head(2)

,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,...,age,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq
Id,,,,,,,,,,,,,,,,,,,,,
ID_2f6873615,NaN,0,5,0,1,1,0,NaN,1,1,...,4,0,16,9,0,1,2.25,0.25,272.25,16
ID_1c78846d2,NaN,0,5,0,1,1,0,NaN,1,1,...,41,256,1681,9,0,1,2.25,0.25,272.25,1681


### Explore Data

#### Information about the data

- One row represents one person.
- Multiple people can be part of a single household. Only predictions for heads of household are scored.

The dataset has observations for each member of the household but the classification is done at the household level. That is, households cannot have two different classifications. Data is not presented at the household level so that participants can create their own household features from individual data.

#### Column descriptions

In [7]:
target_column = 'Target'
person_id = 'Id'
household_id = 'idhogar'
head_of_household = 'parentesco1'

core = {
    person_id: 'a unique identifier for each row',
    target_column: 'the target is an ordinal variable indicating groups of income levels.',
    household_id: 'this is a unique identifier for each household. This can be used to create household-wide features:etc. All rows in a given household will have a matching value for this identifier.',
    head_of_household: 'indicates if this person is the head of the household.'
}

And here's some categories and column descriptions I prepared earlier

In [6]:
from column_categories import target_values, column_descriptions, building_info, \
                                calculated_values, duplicates, individuals_info

There are descriptions for all features in the data:

In [8]:
print_dictionary_head(column_descriptions,3)

Id : a unique identifier for each row
Target : the target is an ordinal variable indicating groups of income levels.
idhogar : Household level identifier


In [9]:
print_dictionary_head(building_info,3)

paredblolad : =1 if predominant material on the outside wall is block or brick
paredzocalo : "=1 if predominant material on the outside wall is socket (wood,  zinc or absbesto"
paredpreb : =1 if predominant material on the outside wall is prefabricated or cement


#### Check for class imbalance

Our target value is a measure per household so we should group households first

In [10]:
def get_household_id(head):
    try:
        return head.loc[head[head_of_household==1]][target_column]
    except KeyError:
        return head[target_column].mode()
    
target_by_household = train.groupby(household_id).apply(get_household_id)

In [11]:
def construct_taget_table_breakdown(series):
    household_target_sizes = series.value_counts().to_frame()
    household_target_sizes.columns = ['total']
    household_target_sizes['proportion'] = household_target_sizes['total']/household_target_sizes['total'].sum()
    household_target_sizes['target description'] = household_target_sizes.index.map(target_values.get)
    return household_target_sizes

construct_taget_table_breakdown(target_by_household)

,total,proportion,target description
4,1956,0.650699,non vulnerable households
2,461,0.153360,moderate poverty
3,375,0.124750,vulnerable households
1,214,0.071191,extreme poverty


What about when we don't breakdown by household (just for comparison in data sample sizes)

In [ ]:
construct_taget_table_breakdown(train[target_column])

Need to do some sort of under- or over-sampling like SMOTE

#### Check dtypes

In [12]:
def get_column_dtypes(df):
    columns_by_dtype = df.columns.groupby(train.dtypes)
    return {k.name: v for k, v in columns_by_dtype.items()}

get_column_dtypes(train)

{'int64': Index(['hacdor', 'rooms', 'hacapo', 'v14a', 'refrig', 'v18q', 'r4h1', 'r4h2',
        'r4h3', 'r4m1',
        ...
        'area1', 'area2', 'age', 'SQBescolari', 'SQBage', 'SQBhogar_total',
        'SQBedjefe', 'SQBhogar_nin', 'agesq', 'Target'],
       dtype='object', length=130),
 'float64': Index(['v2a1', 'v18q1', 'rez_esc', 'meaneduc', 'overcrowding',
        'SQBovercrowding', 'SQBdependency', 'SQBmeaned'],
       dtype='object'),
 'object': Index(['idhogar', 'dependency', 'edjefe', 'edjefa'], dtype='object')}

##### Interpreting object dtypes:

In [13]:
train.select_dtypes('object').head()

,idhogar,dependency,edjefe,edjefa
Id,,,,
ID_279628684,21eb7fcc1,no,10,no
ID_f29eb3ddd,0e5d7a658,8,12,no
ID_68de51c94,2c7317ea8,8,no,11
ID_d671db89c,2b58d945f,yes,11,no
ID_d56d6f5f5,2b58d945f,yes,11,no


###### Inspect and update each object type:

###### `idhogar`

In [14]:
column_descriptions['idhogar']

'Household level identifier'

This is the value we will be using as the ID when training the model, we'll leave data broken down by individual for now while exploring

###### `dependency`

In [15]:
column_descriptions['dependency']

'Dependency rate, calculated = (number of members of the household younger than 19 or older than 64)/(number of member of household between 19 and 64)'

`dependency` needs to be split into boolean and numeric columns

`dependency`, Dependency _rate_, calculated = (number of members of the household younger than 19 or older than 64)/(number of member of household between 19 and 64)

`hogar_nin`+`hogar_mayor` / `hogar_adul`-`hogar_mayor`

Looks like there is a max cutoff of 8

This is related to `SQBdependency`, should that be updated too?

In [16]:
column_descriptions['SQBdependency']

'dependency squared'

In [17]:
train['dependency'].value_counts().sort_index().tail()

5        24
6         7
8       378
no     1747
yes    2192
Name: dependency, dtype: int64

In [18]:
with np.errstate(divide='ignore'):
    train['dependency_calc'] = train[['hogar_nin','hogar_mayor','hogar_adul']].apply(
        lambda row: min(
            (row['hogar_nin']+row['hogar_mayor'])/(row['hogar_adul']-row['hogar_mayor']), 8
        ), axis=1)

Using `np.errstate(divide='ignore')` to ignore issues with division by zero, in which case we will use value 8

Verify that calculation matches what was there before

In [19]:
train[['dependency_calc','dependency']].head(5)

,dependency_calc,dependency
Id,,
ID_279628684,0.0,no
ID_f29eb3ddd,8.0,8
ID_68de51c94,8.0,8
ID_d671db89c,1.0,yes
ID_d56d6f5f5,1.0,yes


The Kaggle competition states that, in the case of inconsistencies for any data withing a household, we always want to take the value given by the household member as the truth 

Let's check for inconsistencies

In [20]:
is_dependency_consistent = train.groupby(household_id)['dependency_calc'].apply(lambda x: x.nunique() == 1)
inconsistent_dependency = is_dependency_consistent[is_dependency_consistent != True]

print(inconsistent_dependency)

Series([], Name: dependency_calc, dtype: bool)


There are no inconsistencies! Woop woop!

###### `edjefe`

In [21]:
column_descriptions['edjefe']

'years of education of male head of household, based on the interaction of escolari (years of education), head of household and gender, yes=1 and no=0'

What kind of values do we have here?

In [22]:
train['edjefe'].value_counts().sort_index()

10      111
11      751
12      113
13      103
14      208
15      285
16      134
17      202
18       19
19       14
2       194
20        7
21       43
3       307
4       137
5       222
6      1845
7       234
8       257
9       486
no     3762
yes     123
Name: edjefe, dtype: int64

Would yes be better represented by the mean number of years? Should we add a boolean column for yes/no values

In [23]:
# yes_value = 1
yes_value = pd.to_numeric(train['edjefe'], errors='coerce').dropna().mean()

In [24]:
train['edjefe'].replace('no','0',inplace=True)
train['edjefe'].replace('yes',yes_value,inplace=True)
train['edjefe'] = train['edjefe'].astype('float')

Let's check for inconsistencies

In [25]:
is_edjefe_consistent = train.groupby(household_id)['edjefe'].apply(lambda x: x.nunique() == 1)
inconsistent_dependency = is_edjefe_consistent[is_edjefe_consistent != True]

print(inconsistent_dependency)

Series([], Name: edjefe, dtype: bool)


Looks consistent!

###### `edjefa`

In [26]:
column_descriptions['edjefa']

'years of education of female head of household, based on the interaction of escolari (years of education), head of household and gender, yes=1 and no=0'

In [27]:
train['edjefa'].value_counts().sort_index()

10       96
11      399
12       72
13       52
14      120
15      188
16      113
17       76
18        3
19        4
2        84
20        2
21        5
3       152
4       136
5       176
6       947
7       179
8       217
9       237
no     6230
yes      69
Name: edjefa, dtype: int64

Same as before

In [28]:
# yes_value = 1
yes_value = pd.to_numeric(train['edjefa'], errors='coerce').dropna().mean()

In [29]:
train['edjefa'].replace('no','0',inplace=True)
train['edjefa'].replace('yes',yes_value,inplace=True)
train['edjefa'] = train['edjefa'].astype('float')

Inconsistency check

In [30]:
is_consistent = train.groupby(household_id)['edjefa'].apply(lambda x: x.nunique() == 1)
inconsistent_dependency = is_consistent[is_consistent != True]

print(inconsistent_dependency)

Series([], Name: edjefa, dtype: bool)


#### Assert Target Values Are Consistent

Check that all members of each household have the same target, in the case of inconsistencies Kaggle has advised to use the Target value given by the head of household.

In [ ]:
def get_inconsistent_rows(df, column_name):
    is_target_consistent = df.groupby(household_id)[column_name].apply(lambda x: x.nunique() == 1)
    inconsistent_targets = is_target_consistent[is_target_consistent != True]
    print(inconsistent_targets.shape)
    return inconsistent_targets

inconsistencies = get_inconsistent_rows(train, target_column)
corrections = train[train[household_id].isin(inconsistencies.index) & (train['parentesco1'] == 1.0)][[household_id,target_column]]
corrections.reset_index().drop(person_id, axis=1)
print(train.shape)

Need to correct 85 inconsistent target values

In [ ]:
updated = train.reset_index().merge(corrections, on=household_id, how='left').set_index(person_id)
updated['Target_x'].update(updated[updated['Target_y'].notnull()]['Target_y'])
train = updated.rename(index=str, columns={'Target_x': target_column}).drop('Target_y', axis=1)

Check we've successfully replaced inconsistent target values

In [ ]:
get_inconsistent_rows(train, target_column)
print(train.shape)

#### Check for features with a lot of missing data

In [ ]:
nulls = train.isnull().sum(axis=0)
nulls[nulls!=0]/len(train)

Compare this to the test data too

In [ ]:
nulls = test.isnull().sum(axis=0)
nulls[nulls!=0]/len(test)

In both these cases `v2a1`,`v18q1`,`rez_esc` are mostly missing.

###### Lookup descriptions

In [ ]:
column_descriptions.get('v2a1')

In [ ]:
column_descriptions.get('v18q1')

In [ ]:
column_descriptions.get('rez_esc')

In [ ]:
column_descriptions.get('meaneduc')

In [ ]:
column_descriptions.get('SQBmeaned')

###### Dig for missing values

`v2a1`,`v18q1`,`meaneduc`,`SQBmeaned` are household level values

How about checking if at least one person in each household might have presented this data so we can use it

In [ ]:
value_missing = ~train.groupby(household_id)['v2a1','v18q1','meaneduc','SQBmeaned'].any()
missing_value_counts = value_missing.sum(axis=0)
missing_value_counts/len(train)

Much fewer values are missing than it originally seemed. Although ~25% missing still feels like quite a lot..

###### Find consistent rows and use first value found

In [ ]:
def get_consistent_rows(df, column_name):
    is_target_consistent = df.groupby(household_id)[column_name].apply(lambda x: x.nunique() == 1)
    consistent_targets = is_target_consistent[is_target_consistent == True]
    return consistent_targets

consistent_rows = get_consistent_rows(train, 'v2a1').to_frame()

In [ ]:
print(train.shape)
updated = train.reset_index().merge(consistent_rows, on=household_id, how='right')
# updated = updated[[household_id,'v2a1_x']].groupby(household_id).first()
updated = updated.rename(index=str, columns={'v2a1_x': 'v2a1'}).drop('v2a1_y', axis=1)
updated.set_index(person_id)
print(updated.head(2))
train.update(updated)

In [ ]:
train.shape